# **주제 : 통신사 이탈예측**
제출일 : 2025-01-26

## **( 1 )데이터 불러오기**

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
df.head(5)

In [ ]:
# 기본 정보 확인
df.info()

In [ ]:
'''
컬럼 정보 확인.
1. customerID (고객ID): 고객 식별 번호
2. gender (성별): 고객의 성별
3. SeniorCitizen (노인여부): 고객이 노인인지 여부 (0: 아님, 1: 노인)
4. Partner (배우자여부): 고객의 배우자 유무
5. Dependents (부양가족여부): 고객의 부양가족 유무
6. tenure (가입기간): 고객이 회사와 거래한 개월 수
7. PhoneService (전화서비스): 전화 서비스 가입 여부
8. MultipleLines (복수회선): 복수 회선 사용 여부
9. InternetService (인터넷서비스): 인터넷 서비스 유형
10. OnlineSecurity (온라인보안): 온라인 보안 서비스 가입 여부
11. OnlineBackup (온라인백업): 온라인 백업 서비스 가입 여부
12. DeviceProtection (기기보호): 기기 보호 서비스 가입 여부
13. TechSupport (기술지원): 기술 지원 서비스 가입 여부
14. StreamingTV (TV스트리밍): TV 스트리밍 서비스 이용 여부
15. StreamingMovies (영화스트리밍): 영화 스트리밍 서비스 이용 여부
16. Contract (계약유형): 계약 유형 (월 단위, 1년, 2년)
17. PaperlessBilling (전자청구서): 전자 청구서 이용 여부
18. PaymentMethod (지불방법): 지불 방법
19. MonthlyCharges (월청구액): 월별 청구 금액
20. TotalCharges (총청구액): 총 청구 금액
21. Churn (이탈여부): 고객 이탈 여부 (목표 변수)
'''

In [ ]:
df.describe()

In [ ]:
# 결측치 확인
print(df.isnull().sum())

## **( 2 )데이터 전처리**

In [ ]:
# 1. customerID 컬럼 삭제
df = df.drop('customerID', axis=1)

In [ ]:
# 2. 범주형 변수 원-핫 인코딩
categorical_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
                    'InternetService', 'OnlineSecurity','OnlineBackup', 'DeviceProtection', 'TechSupport', 
                    'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
df_encoded = pd.get_dummies(df, columns=categorical_cols, dtype=int)

In [ ]:
df_encoded.info()

In [ ]:
# 3. TotalCharges의 공백을 NaN으로 변환 후 결측치 처리
df_encoded['TotalCharges'] = pd.to_numeric(df_encoded['TotalCharges'], errors='coerce')
df_encoded['TotalCharges'] = df_encoded['TotalCharges'].fillna(df_encoded['TotalCharges'].mean())

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 4. tenure, MonthlyCharges, TotalCharges 정규화 처리
scaler = StandardScaler()
df_encoded[['tenure_scaled','MonthlyCharges_scaled', 'TotalCharges_scaled']] = scaler.fit_transform(df_encoded[['tenure', 'MonthlyCharges', 'TotalCharges']])


In [ ]:
# 5. Churn 레이블 인코딩
le = LabelEncoder()
df_encoded['Churn'] = le.fit_transform(df_encoded['Churn'])

In [ ]:
df_encoded.head(5)

## **( 3 ) 데이터 분할**

In [ ]:
from sklearn.model_selection import train_test_split

# 특성(X)과 목표 변수(y) 정의
X = df_encoded.drop('Churn', axis=1)  # 'Churn' 열을 제외한 모든 열을 특성으로 사용
y = df_encoded['Churn']  # 'Churn' 열을 목표 변수로 사용

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

## **( 4 ) 모델 생성 및 학습 : LogisticRegression**
: 로지스틱 회귀 모델이 수렴하지 못했음.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# 로지스틱 회귀 모델 생성 및 학습
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)


In [ ]:
# 예측
y_pred = model.predict(X_test)

# 모델 평가
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

1. 전체 정확도
전체 정확도: 82%
1,409개 중 약 1,158개를 정확히 예측

2. 클래스별 성능
* 0 클래스 (이탈하지 않은 고객)
정밀도: 86%
재현율: 91%
F1 점수: 88%
* 1 클래스 (이탈한 고객)
정밀도: 70%
재현율: 58%
F1 점수: 63%

3. 혼동 행렬 해석
전체 1,409개 중:
* 실제 이탈하지 않은 고객 1,036명
941명 정확히 예측
95명 잘못 예측 (이탈로 잘못 분류)
* 실제 이탈한 고객 373명
217명 정확히 예측
156명 잘못 예측 (이탈하지 않을 것으로 잘못 분류)
* 주요 인사이트
모델은 이탈하지 않은 고객을 더 정확히 예측
이탈 고객 예측에는 상대적으로 어려움 존재
이탈 고객 예측의 정밀도와 재현율이 낮음

## **( 4 ) 모델 생성 및 학습 : LogisticRegression(scaler)**
: 반복 횟수 증가 / 데이터 스케일링 / 솔버 변경

In [ ]:
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# 로지스틱 회귀 모델 생성 및 학습
# max_iter=1000 : 기본값 100에서 반복 횟수 증가
# solver='liblinear' : 작은 데이터셋에 적합

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model = LogisticRegression( random_state=42, max_iter=1000, solver='liblinear' )
model.fit(X_train, y_train)

In [ ]:
# 예측
y_pred = model.predict(X_test)

# 모델 평가
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## **( 4 ) 모델 생성 및 학습 : XGBoost**
: 높은 예측 성능, 빠른 학습

In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)


In [ ]:
y_pred = xgb_model.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## **( 4 ) 모델 생성 및 학습 : RandomForest**
: 과적합 방지, 특성 중요도 제공

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

print(classification_report(y_test, y_pred))

## **( 4 ) 모델 생성 및 학습 : GradientBoosting**
: 복잡한 관계 학습에 우수

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)
y_pred = gb_model.predict(X_test)

print(classification_report(y_test, y_pred))